# TD 3 : Le modèle Leaky Integrate-and-Fire
---

In [ ]:
# import necessary modules for numerics and plotting
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 1. Le modèle Integrate-and-Fire 

La dynamique sous-seuil d'un neurone "Integrate-and-Fire" est décrit par l'équation 
$$\tau \frac{dV}{dt} = -V + RI.$$
Ici, $V$ est le potentiel de la membrane, $\tau$ est la constante de temps de la membrane, $R$ la résistance d'entrée, et $I$ représente un courant injecté dans le neurone. Pour faire simple, tous les potentiels sont mésurés rélatif au potentiel de repos $E_L$, qui pour cette raison n'apparaît plus dans l'équation. Chaque fois que le potentiel excède un seuil $V_{\rm seuil}$, on considère que le neurone emet un potentiel d'action, et le potentiel membranaire est remis à une valeur de "reset", $V\to V_{\rm reset}$.

Considérons les paramètres suivants : $\tau=20$ ms et $R=1$ MOhm pour la dynamique sous-seuil, et $V_{\rm seuil}=20$ mV avec $V_{\rm reset}=0$ mV pour le mécanisme de décharge. 

Remarque : Si nous allons spécifier le courant $I$ en nA, nous pouvons calculer le produit $R*I$ sans dévoir transformer des unités, puisque MOhm x nA = mV. 



## E1.1 Solution analytique de la dynamique sous-seuil du modèle Integrate-and-Fire

Quel est le potentiel de la membrane stationnaire - ou le point fixe - pour $t\rightarrow \infty$ si on considère uniquement la dynamique sous-seuil spécifiée ci-dessus, c'est-à-dire en faisant abstraction du mécanisme de décharge ? 

Vérifiez que $$V(t) = RI + (V_0 - RI) \exp(-t/\tau)$$ est une solution de l'équation différentielle avec la condition initiale $V(t=0) = V_0$. 

**Indice**: Rappelez-vous que $\frac{d}{dt} \left( e^{f(t)} \right) = e^{f(t)} \frac{df(t)}{dt}$

In [ ]:
# calcul et réponses sur papier

In [ ]:
# We can also graphically inspect
# the solution using Python


**Solution :** 

A) 

B) 

## E1.2 L'activité de décharge du neurone Integrate-and-Fire

Dans le modèle Integrate-and-Fire, un potentiel d'action est déclenché lorsque le potentiel de membrane atteint un seuil $V_{\rm seuil}$. Le potentiel est remis à $V \rightarrow V_{\rm reset}$ après le potentiel d'action. 

**Dans quelle condition, c'est-à-dire pour quel courant injecté, le neurone déclenche-t-il des potentiels d'action ?**

Veuillez dériver l'expression analytique du taux de décharge $f(t)$ en fonction de l'input externe $I$. 

**Indice :** Calculez d'abord combien de temps $T$ il faut pour passer de $V(t=0)=V_{\rm reset}$ à $V(T)=V_{\rm seuil}$. Ce temps $T$ entre deux spikes dépend du courant injecté, $T=T(I)$. Le taux de décharge en fonction de l'input est alors donné par $f(I) = 1/T(I)$. Tracez le taux de décharge $f$ en fonction de l'entrée $I$.

In [ ]:
# calcul et réponses sur papier

**Réponses :**

A) 

B) 

In [ ]:
# plot of the firing rate vs. input current


## E1.3 Neurone Integrate-and-Fire avec période refractaire

Considérons maintenant un neurone avec une période réfractaire : Dans cette modification du modèle, le neurone n'intégre pas d'inputs pendant une période brève après l'émission d'un spike, similaire aux vrais neurones. Supposons que la période réfractaire dure un temps $D$. Comment le taux de décharge $f(I)$ change-t-il avec une période réfractaire de $D=5$ ms ? Quelle est la nouvelle expression pour $f(I)$ ? Tracez $f$ en fonction de $I$. 

**Remarque :** La dynamique sous-seuil, une fois la période refractaire passée, est identique au cas sans période refractaire, c'est-à-dire n'est pas modifiée, et un spike est toujours déclenché quand $V>V_{\rm seuil}$.

**Indice :** Dessinez d'abord un croquis de la dynamique du potentiel membranaire pour avoir une meilleure idée de la façon dont $D$ affecte $T$. 

In [ ]:
# calcul et réponses sur papier

**Réponse :**



In [ ]:
# plotting the f-I-curve with refractory period

# (your code)

## E1.4 Intégration numérique du modèle Integrate-and-Fire avec décharge

La semaine précédente, nous avons utilisé la méthode d'Euler pour résoudre des équations différentielles. Nous pouvons utiliser cette même technique pour résoudre la dynamique sous-seuil d'un neurone LIF (Leaky Integrate-and-Fire), en rajoutant un mécanisme de détection de spike et de reset.


In [ ]:
# Discretization of time

# Definition of the dynamics
# (function returning dV/dt)

# Iterative integration of 
# the subthreshold dynamics,
# COMBINED WITH threshold
# crossing detection!

# (your code)

# 2. Activité irrégulière des neurones corticaux

Dans le cortex, le timing des potentiels d'action successifs est très irrégulier. Le coefficient de variation des intervalles inter-spike est proche de 1, similaire à un processus de Poisson. Pour un processus de Poisson, la fonction de densité de probabilité pour le temps d'attente jusqu'au spike suivant est donnée par 
$$p(T) = f {\rm e}^{-f T}$$
Ici, $T$ est l'intervalle interspike, et $f$ est le taux de décharge. 

Ainsi, la densité de l'intervalle interspike pour un train de spike de Poisson homogène est une fonction exponentielle. Les intervalles d'interspike les plus probables sont courts et les intervalles longs ont une probabilité qui diminue de façon exponentielle en fonction de leur durée. Les histogrammes d'intervalles interspike peuvent être extraits des potentiels d'action en calculant la différence de temps entre tous les spikes successifs.  

Dans l'approximation la plus simple, le temps du spike suivant n'est pas du tout correlé avec le temps du spike précédant. (Evidemment, cela n'est pas vrai dès lors qu'il y a un temps refractaire, comme c'est biologiquement impossible qu'un neuron décharge par ex. deux fois d'affilé dans l'espace de moins d'une ms.)

## E2.1  Train de spike Poisson

Générez un train de spikes de Poisson du taux $f = 10$ spk/sec. Calculez la moyenne et l'écart type des intervalles interspike (ISI). Vérifier que le coefficient de variation (CV = écart type/moyenne) est égal à 1. Tracez l'histogramme des ISI. Comment un train de spikes peut-il être généré à partir des ISIs? 


**Indice :** Rappelez-vous que les intervalles d'interspike d'un train de spikes Poisson sont distribués de façon exponentielle.  À son tour, le train peut être généré en tirant des nombres aléatoires de la distribution exponentielle et en les traitant comme des intervalles interspike. Utilisez la fonction numpy `np.random.exponential()` pour tirer les nombres de la distribution exponentielle. 



**Indice :** Vous pouvez créer et plotter des histograms avec la fonction `ax.hist`. 


In [ ]:
# look up the documentation for functions
# that you don't know
np.random.exponential?

In [ ]:
# Sample a distribution of exponentially 
# distributed ISIs respecting that the 
# given average firing rate

# (your code)

In [ ]:
# Plot the histogram

# (your code)

In [ ]:
# Plot the spiketrain
# TIP: use ax.vlines to plot
# vertical lines at a given 
# value along the x-axis, such
# that each spike is represented
# by a line

# (your code)

---
## Bonus :

# 3. Décharge irrégulière dans le modèle Integrate-and-Fire

Ajoutons une entrée de bruit à l'équation du neurone Integrate-and-Fire ci-dessus
$$\tau \frac{dV}{dt} = -V + RI + \sigma \sqrt{\tau} \xi(t),$$
où $\sigma$ détermine l'amplitude du bruit et $\xi(t)$ est un processus de bruit blanc gaussien avec une densité de variance unitaire et moyenne zéro.

Le terme de bruit peut par ex. représenter :     
* la stochasticité d'ouverture et de fermeture des canaux ioniques,
* le bruit synaptique (fiabilité de la transmission synaptique),
* les entrées synaptiques externes (inconnues et/ou variables entre les essais).

Il est implémenté comme suit dans un schéma d'intégration numérique: 
 1. Tirez un nombre aléatoire $\eta$ pour chaque pas de temps à partir d'une distribution gaussienne avec une moyenne zéro et une variance unitaire. Ceci peut être réalisé en utilisant la fonction numpy suivante: `np.random.normal()`.
 2. Échellez ce nombre par la largeur de l'intervalle de temps : $\xi = \eta/\sqrt{dt}$
 3. Utilisez ce nombre dans l'équation différentielle. 


## E3.1 Intégration numérique d'une équation différentielle stochastique : Integrate-and-Fire avec le bruit

Écrivez un schéma d'intégration numérique de l'équation différentielle ci-dessus. Utilisez la méthode Euler : $x(t+\Delta t) = x(t) +  \Delta t (dx/dt)$ avec un largeur de pas de temps de $0.1$ ms et intégrez pendant 20 secondes. Enregistrez les temps de spikes et calculez le coefficient de variation après l'intégration. Essayez d'obtenir un $CV$ de 1 à un taux de décharge de 1 spk/sec en variant $I$ et $\sigma$. Tracer le potentiel de la membrane pendant une période de 20 sec. Inspectez l'histogramme de la distribution ISI.  



In [ ]:
# Your code